In [1]:
from aocd import get_data
from aocd import submit
import unittest

day = 10
year = 2023

def submit_part_a(answer):
    submit(answer, part="a", day=day, year=year)

def submit_part_b(answer):
    submit(answer, part="b", day=day, year=year)

input = get_data(day=day, year=year)


In [2]:
north = (0, -1) # (0, -1) means (x, y-1) means north
south = (0, 1)  # (0, 1)  means (x, y+1) means south
west  = (-1, 0) # (-1, 0) means (x-1, y) means west
east  = (1, 0)  # (1, 0)  means (x+1, y) means east

north_pipes = ['|', 'F', '7']
south_pipes = ['|', 'L', 'J']
west_pipes  = ['-', 'L', 'F']
east_pipes  = ['-', 'J', '7']

# 'current_pipe' : {
#     direction : [
#         'first_possible_pipe_to_make_connection',
#         'second_possible_pipe_to_make_connection',
#         'thirs_possible_pipe_to_make_connection'
#     ]
# }
pipes_connections = {
    '-' : {west:  west_pipes   , east : east_pipes},
    '|' : {north: north_pipes  , south : south_pipes},
    'F' : {east : east_pipes   , south: south_pipes},
    'L' : {north: north_pipes  , east : east_pipes},
    'J' : {north: north_pipes  , west: west_pipes},
    '7' : {west:  west_pipes   , south: south_pipes}
}

In [3]:
def sum_tuples(tuple1, tuple2):
    return tuple(map(sum, zip(tuple1, tuple2)))

In [4]:
pipes = [[c for c in list(line)] for line in input.split('\n')]

# find start_position
start_position = (0,0)
for y in range(len(pipes)):
    for x in range(len(pipes[y])):
        if pipes[y][x] == 'S':
            start_position = (x, y)

# find pipe of start_position
up = sum_tuples(start_position, north)
down = sum_tuples(start_position, south)
left = sum_tuples(start_position, west)
right = sum_tuples(start_position, east)

possible_start_pipes = []
if pipes[up[1]][up[0]] in north_pipes:
    # possible match for connecting start_pipe to north
    # so possible list of possible start pipes is south_pipes
    possible_start_pipes += south_pipes
if pipes[down[1]][down[0]] in south_pipes:
    # possible match for connecting start_pipe to south
    # so possible list of possible start pipes is north_pipes
    possible_start_pipes += north_pipes
if pipes[left[1]][left[0]] in east_pipes:
    # possible match for connecting start_pipe to east
    # so possible list of possible start pipes is west_pipes
    possible_start_pipes += west_pipes
if pipes[right[1]][right[0]] in west_pipes:
    # possible match for connecting start_pipe to west
    # so possible list of possible start pipes is east_pipes
    possible_start_pipes += east_pipes

# The start_pipe is the one we found twice in possible_start_pipes
start_pipe = [pipe for pipe in set(possible_start_pipes) if possible_start_pipes.count(pipe) == 2][0]
pipes[start_position[1]][start_position[0]] = start_pipe

print(f"start position is (x, y) = {start_position}")
print(f"start pipe is = {start_pipe}")

start position is (x, y) = (8, 42)
start pipe is = F


In [5]:
def next_position_for(pipes, from_position, visited_positions):
    pipe = pipes[from_position[1]][from_position[0]]
    for direction, connected_pipes in pipes_connections[pipe].items():
        next_position = sum_tuples(from_position, direction)
        if next_position not in visited_positions and pipes[next_position[1]][next_position[0]] in connected_pipes:
            return next_position

def explore_pipes(pipes, from_position):
    # as we're riding the pipes in two different ways at once
    current_position_1 = from_position
    current_position_2 = from_position
    visited_positions =[]
    visited_positions.append(from_position)
    while current_position_1 is not None and current_position_2 is not None :
        # find the path to follow from current_position_1
        current_position_1 = next_position_for(pipes, current_position_1, visited_positions)
        visited_positions.append(current_position_1)
        
        # find the path to follow from current_position_2
        current_position_2 = next_position_for(pipes, current_position_2, visited_positions)
        visited_positions.append(current_position_2)
    visited_positions.remove(None)
    
    return visited_positions

example_pipes = [
    ['F', 'F', '7', 'F', '7', 'F', '7', 'F', '7', 'F', '7', 'F', '7', 'F', '7', 'F', '-', '-', '-', '7'],
    ['L', '|', 'L', 'J', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', '|', 'F', '-', '-', 'J'],
    ['F', 'L', '-', '7', 'L', 'J', 'L', 'J', '|', '|', '|', '|', '|', '|', 'L', 'J', 'L', '-', '7', '7'],
    ['F', '-', '-', 'J', 'F', '-', '-', '7', '|', '|', 'L', 'J', 'L', 'J', '7', 'F', '7', 'F', 'J', '-'],
    ['L', '-', '-', '-', 'J', 'F', '-', 'J', 'L', 'J', '.', '|', '|', '-', 'F', 'J', 'L', 'J', 'J', '7'],
    ['|', 'F', '|', 'F', '-', 'J', 'F', '-', '-', '-', '7', 'F', '7', '-', 'L', '7', 'L', '|', '7', '|'],
    ['|', 'F', 'F', 'J', 'F', '7', 'L', '7', 'F', '-', 'J', 'F', '7', '|', 'J', 'L', '-', '-', '-', '7'],
    ['7', '-', 'L', '-', 'J', 'L', '7', '|', '|', 'F', '7', '|', 'L', '7', 'F', '-', '7', 'F', '7', '|'],
    ['L', '.', 'L', '7', 'L', 'F', 'J', '|', '|', '|', '|', '|', 'F', 'J', 'L', '7', '|', '|', 'L', 'J'],
    ['L', '7', 'J', 'L', 'J', 'L', '-', 'J', 'L', 'J', 'L', 'J', 'L', '-', '-', 'J', 'L', 'J', '.', 'L']
]

example_visited_positions = explore_pipes(example_pipes, (4, 0))
print(int((len(example_visited_positions)) / 2))

80


In [6]:
%%time
visited_positions = explore_pipes(pipes, start_position)
first_answer = int((len(visited_positions)) / 2)
print(first_answer)

6838
CPU times: user 2.28 s, sys: 5.99 ms, total: 2.29 s
Wall time: 2.29 s


In [7]:
submit_part_a(first_answer)

aocd will not submit that answer again. At 2023-12-10 04:56:50.532349-05:00 you've previously submitted 6838 and the server responded with:
That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


In [8]:
# cleanup the pipes to remove all pipes that are not part of the loop
def cleanup(pipes, visited_positions):
    for y in range(len(pipes)):
        for x in range(len(pipes[y])):
            if (x, y) not in visited_positions:
                pipes[y][x] = '.'

cleanup(example_pipes, example_visited_positions)
for line in example_pipes:
    print(''.join(line))

.F7F7F7F7F7F7F7F---7
.|LJ||||||||||||F--J
.L-7LJLJ||||||LJL-7.
F--JF--7||LJLJ.F7FJ.
L---JF-JLJ....FJLJ..
...F-JF---7...L7....
..FJF7L7F-JF7..L---7
..L-JL7||F7|L7F-7F7|
.....FJ|||||FJL7||LJ
.....L-JLJLJL--JLJ..


In [9]:
def are_pipes_connected(pipe1, direction, pipe2):
    if pipe1 == '.' or direction not in pipes_connections[pipe1].keys():
        return False
    return pipe2 in pipes_connections[pipe1][direction]

def extends_pipes(pipes):
    nb_lines = len(pipes)
    nb_cols = len(pipes[0])
    
    # extends the pipes to allow water to go between pipes
    # Add one line of point at the begining (first line)
    extended_pipes = [['.'] * (nb_cols + 1) * 2]
    for y in range(len(pipes)):
        # Add one point at the begining
        original_line_extended = ['.']
        additional_line_extended = ['.']
    
        for x in range(nb_cols):
            current_pipe = pipes[y][x]
            original_line_extended.append(current_pipe)
            
            if y+1 < nb_lines and are_pipes_connected(current_pipe, south, pipes[y+1][x]):
                additional_line_extended.append("|")
            else:
                additional_line_extended.append('.')
            
            if x+1 < nb_cols and are_pipes_connected(current_pipe, east, pipes[y][x+1]):
                original_line_extended.append('-')
                additional_line_extended.append('.')
            else:
                original_line_extended.append('.')
                additional_line_extended.append('.')
        
        # Add one point at the end
        original_line_extended.append('.')
        additional_line_extended.append('.')
                    
        extended_pipes.append(original_line_extended)
        extended_pipes.append(additional_line_extended)
    return extended_pipes 

extended_example_pipes = extends_pipes(example_pipes)
for line in extended_example_pipes:
    print(''.join(line))

..........................................
...F-7.F-7.F-7.F-7.F-7.F-7.F-7.F-------7..
...|.|.|.|.|.|.|.|.|.|.|.|.|.|.|.......|..
...|.L-J.|.|.|.|.|.|.|.|.|.|.|.|.F-----J..
...|.....|.|.|.|.|.|.|.|.|.|.|.|.|........
...L---7.L-J.L-J.|.|.|.|.|.|.L-J.L---7....
.......|.........|.|.|.|.|.|.........|....
.F-----J.F-----7.|.|.L-J.L-J...F-7.F-J....
.|.......|.....|.|.|...........|.|.|......
.L-------J.F---J.L-J.........F-J.L-J......
...........|.................|............
.......F---J.F-------7.......L-7..........
.......|.....|.......|.........|..........
.....F-J.F-7.L-7.F---J.F-7.....L-------7..
.....|...|.|...|.|.....|.|.............|..
.....L---J.L-7.|.|.F-7.|.L-7.F---7.F-7.|..
.............|.|.|.|.|.|...|.|...|.|.|.|..
...........F-J.|.|.|.|.|.F-J.L-7.|.|.L-J..
...........|...|.|.|.|.|.|.....|.|.|......
...........L---J.L-J.L-J.L-----J.L-J......
..........................................


In [10]:
def get_positions_to_flood(pipes, current_position):
    x, y = current_position
    next_positions_to_flood = []
    nb_lines = len(pipes)
    nb_cols = len(pipes[0])
    if y   > 0         and   pipes[y-1][x] == '.':
        next_positions_to_flood.append((x, y-1))
    if y+1 < nb_lines  and   pipes[y+1][x] == '.':
        next_positions_to_flood.append((x, y+1))
    if x > 0           and   pipes[y][x-1] == '.':
        next_positions_to_flood.append((x-1, y))
    if x+1 < nb_cols   and   pipes[y][x+1] == '.':
        next_positions_to_flood.append((x+1, y))
    return next_positions_to_flood

def fulfil_with_water(pipes):
    # start to flood from four corners to center
    positions_to_flood = [
        (0, 0),
        (0, len(pipes)-1),
        (len(pipes[0])-1, len(pipes)-1),
        (len(pipes[0])-1, 0)
    ]
    # while there is position to flood
    while positions_to_flood:
        new_positions_to_flood = []
        for x, y in positions_to_flood:
#            print(x, y)
            pipes[y][x] = ' '
            new_positions_to_flood += get_positions_to_flood(pipes, (x, y))
        positions_to_flood.clear()
        positions_to_flood += set(new_positions_to_flood)

fulfil_with_water(extended_example_pipes)
for l in extended_example_pipes:
    print(''.join(l))

                                          
   F-7 F-7 F-7 F-7 F-7 F-7 F-7 F-------7  
   |.| |.| |.| |.| |.| |.| |.| |.......|  
   |.L-J.| |.| |.| |.| |.| |.| |.F-----J  
   |.....| |.| |.| |.| |.| |.| |.|        
   L---7.L-J.L-J.| |.| |.| |.L-J.L---7    
       |.........| |.| |.| |.........|    
 F-----J.F-----7.| |.L-J.L-J...F-7.F-J    
 |.......|     |.| |...........| |.|      
 L-------J F---J.L-J.........F-J L-J      
           |.................|            
       F---J.F-------7.......L-7          
       |.....|       |.........|          
     F-J.F-7.L-7 F---J.F-7.....L-------7  
     |...| |...| |.....| |.............|  
     L---J L-7.| |.F-7.| L-7.F---7.F-7.|  
             |.| |.| |.|   |.|   |.| |.|  
           F-J.| |.| |.| F-J.L-7 |.| L-J  
           |...| |.| |.| |.....| |.|      
           L---J L-J L-J L-----J L-J      
                                          


In [11]:
def reduces_pipes(pipes):
    nb_lines = len(pipes)
    nb_cols = len(pipes[0])

    reduced_pipes = []
    # reduces the pipes to cancel the extending process water to go between pipes
    # As reminder, I added one line at begining and one line at end
    #              I also added one col and one line between each lines and cols
    for y in range(1, nb_lines, 2):
        reduced_line = []
        for x in range(1, nb_cols, 2):
            reduced_line.append(pipes[y][x])
        reduced_pipes.append(reduced_line)
        
    return reduced_pipes

reduced_example_pipes = reduces_pipes(extended_example_pipes)
for line in reduced_example_pipes:
    print(''.join(line))

 F7F7F7F7F7F7F7F---7 
 |LJ||||||||||||F--J 
 L-7LJLJ||||||LJL-7  
F--JF--7||LJLJ.F7FJ  
L---JF-JLJ....FJLJ   
   F-JF---7...L7     
  FJF7L7F-JF7..L---7 
  L-JL7||F7|L7F-7F7| 
     FJ|||||FJL7||LJ 
     L-JLJLJL--JLJ   


In [12]:
def count_nb_points(pipes):
    nb_points = 0
    for y in range(len(pipes)):
        nb_points += pipes[y].count('.')
    return nb_points

print(count_nb_points(reduced_example_pipes))

10


In [13]:
%%time
cleanup(pipes, visited_positions)
extended_pipes = extends_pipes(pipes)
fulfil_with_water(extended_pipes)
reduced_pipes = reduces_pipes(extended_pipes)
second_answer = count_nb_points(reduced_pipes)
print(second_answer)

451
CPU times: user 2.89 s, sys: 8.39 ms, total: 2.9 s
Wall time: 2.9 s


In [14]:
submit_part_b(second_answer)

aocd will not submit that answer again. At 2023-12-10 12:28:51.050376-05:00 you've previously submitted 451 and the server responded with:
That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 10! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Post-Mortem
It was possible to use the ray-casting algorithm that consists in counting the number of pipes between a point an a border.
If the number of pipes is pair then the point is outside the main loop.
If the number of pipes is unpair then the point is inside the main loop.

It's like looking if parenthesis are well formed (we have exactly the same number of '(' than ')').

The particularity in this case is that:
- if we encounter `L---J` or `F---7` we have to consider there is no pipe
- if we encounter `F---J` or `L---7` we have to consider there is one pipe no mater the unmber of '-' that is between "angles" (`L`, `J`, `7`, `F`)


More information here: https://en.wikipedia.org/wiki/Point_in_polygon
